In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time
from bs4 import BeautifulSoup
import requests

In [2]:
driver = webdriver.Chrome()
driver.maximize_window()

In [ ]:
parent_url = "https://www.imdb.com/search/title/?title_type=feature,tv_movie&release_date=2024-01-01,2024-12-31&country_of_origin=IN&primary_language=hi&adult=include&sort=release_date,asc"

driver.get(parent_url)
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'}
response = requests.get(parent_url, headers=headers)
main_soup = BeautifulSoup(response.text, "html.parser")

total_movies = main_soup.select("div.fwjHEn")[0].get_text(strip=True).split()[-1]

<html>
<head><title>403 Forbidden</title></head>
<body>
<center><h1>403 Forbidden</h1></center>
</body>
</html>



In [39]:
movie_list = []

for i in range(1, int(total_movies)+1):
    if (i % 50 == 0):
        see_more_button = "(//span[@class='ipc-see-more__text'])"

        try:
            button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, see_more_button))
            )

            driver.execute_script("arguments[0].click()", button)
        
        except Exception as e:
            print("Element not found or clickable!")
            time.sleep(2)

    thumbnail = main_soup.select("li.ipc-metadata-list-summary-item")[i-1]

    if thumbnail.select_one(".ipc-media--fallback") is not None:
        print("Thumbnail not found!")
        pass
    else:
        xpath = f"(//a[@class='ipc-title-link-wrapper'])[{i}]"

        try:
            button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, xpath))
            )

            driver.execute_script("arguments[0].click()", button)
            
        except Exception as e:
            print("Element not found or clickable!")
            time.sleep(2)

        current_url = driver.current_url
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'}
        response = requests.get(current_url, headers=headers)
        sub_page_soup = BeautifulSoup(response.text, "html.parser")

        try:
            title = sub_page_soup.select("span.hero__primary-text")[0].get_text(strip=True)
        except IndexError:
            title = "NA"

        try:            
            duration = sub_page_soup.select("ul.joVhBE > li:nth-of-type(2)")[0].get_text(strip=True)
        except IndexError:
            duration = "NA"
            
        try:
            imdb_rating = sub_page_soup.select("span.imUuxf")[0].get_text(strip=True)
        except IndexError:
            imdb_rating = "NA"

        try:
            director_links = sub_page_soup.select('a[href*="ref_=tt_cst_dr_"]')
            directors_set = set(link.get_text(strip=True) for link in director_links)
            directors = list(directors_set)
        except IndexError:
            directors = "NA"

        try:
            writer_links = sub_page_soup.select('a[href*="ref_=tt_ov_wr_"]')
            writers_set = set(link.get_text(strip=True) for link in writer_links)
            writers = list(writers_set)
        except IndexError:
            writers = "NA"

        try:
            cast_links = sub_page_soup.select("a.kVdWAO")
            cast = list(link.get_text(strip=True) for link in cast_links)
        except IndexError:
            cast = "NA"

        try:
            genre_links = sub_page_soup.select('a[href*="ref_=tt_ov_in_"]')
            genres = list(link.get_text(strip=True) for link in genre_links)
        except IndexError:
            genres = "NA"

        try:
            release_date = sub_page_soup.select_one('a.ipc-metadata-list-item__list-content-item[href*="ref_=tt_dt_rdat"]').get_text(strip=True)
        except AttributeError:
            release_date = "NA"

        try:
            language_links = sub_page_soup.select('a[href*="ref_=tt_dt_ln"]')
            languages = list(link.get_text(strip=True) for link in language_links)
        except IndexError:
            languages = "NA"

        try:
            production_company_links = sub_page_soup.select('a[href*="ref_=tt_dt_cmpy_"]')
            production_companies = list(link.get_text(strip=True) for link in production_company_links)
        except IndexError:
            production_companies = "NA"

        try:
            gross_worldwide = sub_page_soup.select('li.ipc-inline-list__item > span.ipc-metadata-list-item__list-content-item')[0].get_text(strip=True)
        except IndexError:
            gross_worldwide = "NA"
            

        movie_details = {"title":title,
                        "duration":duration,
                        "imdb_rating":imdb_rating, 
                        "directors":directors,
                        "writers":writers,
                        "cast":cast,
                        "genres":genres,
                        "release_date":release_date,
                        "languages":languages,
                        "production_companies":production_companies,
                        "gross_worldwide":gross_worldwide}

        movie_list.append(movie_details)

        time.sleep(2)
        driver.back()


Thumbnail not found!


In [40]:
movie_list

[{'title': 'Tapori Nayak',
  'duration': 'NA',
  'imdb_rating': '8.7',
  'directors': [],
  'writers': [],
  'cast': ['Caroline Cave', 'Neil Grayston', 'Nitish Nirmal'],
  'genres': ['Adventure'],
  'release_date': 'January 4, 2024 (India)',
  'languages': ['Hindi'],
  'production_companies': [],
  'gross_worldwide': '₹1,000,000 (estimated)'}]

In [41]:
driver.close()